# Generate a human readable report of all questions and bot responses from all four bots

In [2]:
import pandas as pd
import json, re

MODELS = ["gpt-3.5-turbo", "gpt-4", "gpt-4o", "insolvency_bot_with_gpt-3.5-turbo", "insolvency_bot_with_gpt-4", "insolvency_bot_with_gpt-4o"]
DATASETS = ["train", "test"]

for DATASET in DATASETS:
    df_all_data = pd.DataFrame()
    for MODEL in MODELS:    
        df = pd.read_csv(f"scores_{DATASET}_{MODEL}.csv", encoding="utf-8", sep="\t")
        # Drop the TOTAL row
#         df = df[df.question_no != "TOTAL"]
        
        if MODEL == "gpt-3.5-turbo":
            df_all_data["question_no"] = df['question_no']
            df_all_data["question_text"] = df['question_text']            
            df_all_data["max_points_available"] = df['max_points_available']
            df_all_data["mark_scheme"] = df['mark_scheme']
        
        
        df_all_data["response_" + MODEL] = df['bot_response']
        df_all_data["score_breakdown_" + MODEL] = df['bot_score_breakdown']
        df_all_data["score_" + MODEL] = df['bot_score']
        
        
    df_all_data = df_all_data[df_all_data.max_points_available > 0]    
    
    with open(f"all_responses_{DATASET}.md", "w", encoding="utf-8") as f:
        f.write(f"# All responses for {DATASET} questions\n\n")
        for idx in range(len(df_all_data)):
            if df_all_data.question_no.iloc[idx] == "TOTAL":
                continue
            f.write(f"## Question {re.sub(r'Q', '', df_all_data.question_no.iloc[idx])}\n\n")
            f.write(f"*Text of {df_all_data.question_no.iloc[idx]}*: {df_all_data.question_text.iloc[idx]}\n\n")
            f.write(f"*Maximum points available for {df_all_data.question_no.iloc[idx]}*: {df_all_data.max_points_available.iloc[idx]}\n\n")
            f.write(f"*Mark scheme {df_all_data.question_no.iloc[idx]}*\n\n")
            ms = json.loads(df_all_data.mark_scheme.iloc[idx])

            mst = pd.DataFrame()
            mst["no"] = list(range(1, len(ms[0]) + 1))
            mst["points"] = ms[1]
            mst["criterion"] = ms[0]

            f.write(mst.to_markdown(index=False) +"\n\n")

            for MODEL in MODELS:
                sc = df_all_data["score_" + MODEL].iloc[idx]
                f.write(f"### {MODEL} response to {df_all_data.question_no.iloc[idx]} (score: {sc})\n\n")

                f.write(df_all_data["response_" + MODEL].iloc[idx] + "\n\n")

            mst.drop(columns=["criterion"], axis=1, inplace=True)
            mst.rename(columns={"points":"max points"}, inplace=True)
            f.write(f"### Scores of all four models on {df_all_data.question_no.iloc[idx]}\n\n")
            for MODEL in MODELS:
                mst[MODEL] = json.loads(df_all_data["score_breakdown_" + MODEL].iloc[idx])

            f.write(mst.to_markdown(index=False) +"\n\n")

/home/thomas/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [5]:
df_all_data.to_excel("score_summary_2.xlsx", index=False)